In [1]:
import numpy as np
import os
import multiprocessing as mul
import matplotlib.pyplot as plt
from numba import jit, njit, prange, set_num_threads, vectorize, guvectorize, cuda
from tqdm import tqdm
import shutil

shutil.copytree("./o_data/icecube_10year_ps/events/", "./data/icecube_10year_ps/events/", dirs_exist_ok=True)

from core.signal_bag import *
from core.stacking_analysis import *
from core.req_arrays import *
import pandas as pd
import pickle
import scipy.stats as st
import scipy.interpolate as interp

read icdata
read uptdata
read eadata
read mspdata


### GENERATING SYNTHETIC ICECUBE

# Angerr

smear = pd.read_csv('data/icecube_10year_ps/irfs/IC59_smearing.csv', sep='\s+', skiprows=1, names='log10(E_nu/GeV)_min	log10(E_nu/GeV)_max	Dec_nu_min[deg]	Dec_nu_max[deg]	log10(E/GeV)_min	log10(E/GeV)_max	PSF_min[deg]	PSF_max[deg]	AngErr_min[deg]	AngErr_max[deg]	Fractional_Counts'.split('\t'))

loge_bins_smeared = np.linspace(2, 9, 15)
decl_bins_smeared = np.array([-90, -10, 10, 90], dtype=np.float64)


angerr_smear_max_DICT = {}
angerr_smear_min_DICT = {}

for i in loge_bins_smeared[:-1]:
    angerr_smear_max = []
    angerr_smear_min = []
        
    for j in decl_bins_smeared[:-1]:
        temp = smear[smear['log10(E_nu/GeV)_min'] == i]
        angerr_smear_max.append(temp[temp['Dec_nu_min[deg]'] == j]['AngErr_max[deg]'].max())
        angerr_smear_min.append(temp[temp['Dec_nu_min[deg]'] == j]['AngErr_min[deg]'].min())
        
    angerr_smear_max_DICT[i] = angerr_smear_max
    angerr_smear_min_DICT[i] = angerr_smear_min

In [2]:
season = 0
nbins = 15
enus = np.logspace(11.001, 18.999, int(nbins))
enus_bin_indices = np.zeros(len(enus), dtype=np.int64)

for i in prange(len(enus)):
    enus_bin_indices[i] = np.digitize(enus[i], e_nu_wall) - 1
msdec_bin_indices = np.zeros(p, dtype=np.int64)
for i in prange(p):
    msdec_bin_indices[i] = np.digitize(msdec[i], dec_nu) - 1

for i in prange(len(enus)):
    enus_bin_indices[i] = np.digitize(enus[i], e_nu_wall) - 1
gamma_arr = [-2, -2.2, -2.53, -3]
phio = np.logspace(-38, -26, 1000) #CHANGING TO LINEAR BINS RESULTS IN STRAIGHT LINES

In [3]:
syn_nu_choice = np.random.randint(0, p, 50) #Choose 50 random pulsars from the 3389 pulsars
syn_nudec_bin = msdec_bin_indices[syn_nu_choice] #Find the declination bin of the chosen pulsars to be allocated for the synthetic neutrinos
syn_nu_ra = msra[syn_nu_choice] #Find the right ascension of the chosen pulsars to be allocated for the synthetic neutrinos
syn_nu_dec = msdec[syn_nu_choice] #Find the declination of the chosen pulsars to be allocated for the synthetic neutrinos
phio_const = 4.98 * (10**(-27)) #GeV-1 to ev-1 conversion factor
filenames = ["IC40_exp.csv", "IC59_exp.csv","IC79_exp.csv", "IC86_I_exp.csv", "IC86_II_exp.csv",
        "IC86_III_exp.csv", "IC86_IV_exp.csv", "IC86_V_exp.csv", "IC86_VI_exp.csv", "IC86_VII_exp.csv"]

i=0
j=40
N_nu_k = t_upt[season] * earea[season][syn_nudec_bin[j] * 40 + enus_bin_indices[i]] * enus[i] * phio_const * ((enus[i] / 10**14) ** gamma_arr[2])

In [11]:

# syn_nu_dec = []
syn_N_nu = 0   # No.of neutrinos generated per season per energy bin per declination bin
for season_i in range(10):
    icdata_k = pd.read_csv("./o_data/icecube_10year_ps/events/" + filenames[season_i], sep="\s+", comment="#", names="MJD[days]	log10(E/GeV)	AngErr[deg]	RA[deg]	Dec[deg]	Azimuth[deg]	Zenith[deg]".split("\t"), dtype=float)
    min_ang_err_k = min(icdata_k['AngErr[deg]'])
    max_ang_err_k = max(icdata_k['AngErr[deg]'])
    syn_N_nu_sing_season = 0
    c= 0
    for i in range(len(enus)):    
    
        # apt = 0
        for j in range(len(syn_nudec_bin)):
            n_nu_temp = t_upt[season_i] * earea[ea_season(season_i)][syn_nudec_bin[j] * 40 + enus_bin_indices[i]] * enus[i] * phio_const * ((enus[i] / 10**14) ** gamma_arr[2])
            #n_nu_temp neutrinos are generated in this season, in this energy bin, in this declination bin
            n_nu_i = 0
            for n_nu_i in range(int(np.round(n_nu_temp, 0))):
                
                
                tempp = pd.DataFrame([-1, -1, np.random.uniform(0.2, 15), msra[syn_nu_choice[j]], msdec[syn_nu_choice[j]], -1, -1]).T
                tempp.columns = "MJD[days]	log10(E/GeV)	AngErr[deg]	RA[deg]	Dec[deg]	Azimuth[deg]	Zenith[deg]".split("\t")
                icdata_k = pd.concat([icdata_k, tempp], ignore_index=True )
                c+=1
            # apt+=(n_nu_temp)
            syn_N_nu_sing_season+=n_nu_i
            
            
            
            
        # syn_N_nu_sing_season+=apt
    # icdata_k.to_csv("./data/icecube_10year_ps/events/" + filenames[season_i], sep="\t", index=False)
    print(len(icdata_k.index), syn_N_nu_sing_season, icwidths[season_i + 1], c, len(icdata_k.index) - icwidths[season_i + 1])    
    syn_N_nu += syn_N_nu_sing_season

38496 1421 36900 1596 1596
109906 2693 107011 2895 2895
96657 3309 93133 3524 3524
140515 4012 136244 4271 4271
116979 3905 112858 4121 4121
126788 4029 122541 4247 4247
131525 4259 127045 4480 4480
133671 4142 129311 4360 4360
127688 3815 123657 4031 4031
150399 4428 145750 4649 4649


In [12]:
icdata_k

,MJD[days],log10(E/GeV),AngErr[deg],RA[deg],Dec[deg],Azimuth[deg],Zenith[deg]
0,57891.176944,2.90,0.890000,343.245000,13.768000,46.287,103.860
1,57891.180951,3.13,0.350000,127.003000,35.576000,263.912,125.521
2,57891.188775,4.70,0.200000,216.473000,-23.266000,177.292,66.652
3,57891.195600,3.02,3.260000,176.624000,31.430000,219.626,121.334
4,57891.195746,3.26,0.380000,42.288000,4.950000,354.022,95.023
...,...,...,...,...,...,...,...
150394,-1.000000,-1.00,11.994025,284.110833,4.073889,-1.000,-1.000
150395,-1.000000,-1.00,4.482297,284.900000,3.216667,-1.000,-1.000
150396,-1.000000,-1.00,2.549661,254.287917,-4.100278,-1.000,-1.000
150397,-1.000000,-1.00,9.299683,161.250000,-4.000000,-1.000,-1.000


In [1]:
import sys

In [14]:
sys.argv(100)
exec(open("./task4s_sim_preprocess.py").read())

TypeError: 'list' object is not callable

In [15]:
print(sys.argv)

['/home/darkwake/.local/lib/python3.10/site-packages/ipykernel_launcher.py', '--ip=127.0.0.1', '--stdin=9003', '--control=9001', '--hb=9000', '--Session.signature_scheme="hmac-sha256"', '--Session.key=b"42cde5c8-2510-414d-84cd-b3d9b18ca51d"', '--shell=9002', '--transport="tcp"', '--iopub=9004', '--f=/home/darkwake/.local/share/jupyter/runtime/kernel-v2-36252AXM633A8U5Yr.json']


In [16]:
import subprocess

In [ ]:
subprocess.call([sys.executable, 'task4s_sim_preprocess.py', '-nb', '100'])

In [2]:
sys.executable

'/home/darkwake/anaconda3/bin/python'